## Proposal_Differentiation_Tool

Differentiate between Titles, Headers, Body paragraphs, etc.

Should we create a separate excel file to track which proposals are "good" (won)?

### Initialize

In [1]:
import fitz
import json
from re import findall

### Get All Text tool

In [98]:
# Select document & page number (0-indexed) to read
doc = fitz.open("./downloaded/PDF.pdf")
page_num = 4

# Run function to get the plain text
get_pdf_text(doc, page_num)

mupdf: cannot open ./downloaded/PDF.pdf: No such file or directory


RuntimeError: cannot open ./downloaded/PDF.pdf: No such file or directory

In [96]:
def get_pdf_text(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Group json entries by those that share the same color&font&size

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    
    # Set previous properties to None to prepare for comparisons
    prev_color = None
    prev_font = None
    prev_size = None
    text_holder = ""

    # Iterate through all the text entries
    for entry in text_blocks:
        # Check if this block is an image
        try:
            img = entry["image"]
        except:
            img = None
            pass

        # Check if this block exists, or if the end of the page has been reached
        try: 
            lines = entry["lines"]
        except:
            lines = None
            pass

        # Line is not an image and contains text so we continue
        if lines != None and img == None:             
            for spans in lines:
                # Get the line data
                spans = spans["spans"]                

                for i in range(len(spans)):
                    span = spans[i]
                    
                    # Identify current line's properties and text
                    font = span["font"]
                    color = span["color"]
                    size = span["size"]
                    text = span["text"]

                    # Check if current line has different properties from previous
                    if (font != prev_font) or (color != prev_color) or (size != prev_size):
                        # Current line has different properties.

                        # For now, just print whatever text had the previous properties
                        last_printed_text = text_holder
                        print(text_holder, "\n")

                        # Reset text holder & prev_ properties
                        text_holder = text
                        prev_font = font
                        prev_size = size
                        prev_color = color

                    else:
                        # Current line has the same properties. Add it to the text block.
                        text_holder += text
                        
    if last_printed_text != text_holder:
        print(text_holder)
  
    return

### Differentiation between Text and Headers

In [71]:
%%time
# Input document
doc = fitz.open("./downloaded/PDF.pdf")

# Check if Table of Contents (bookmarks) exists for the PDF
toc = doc.getToC()
if len(toc) != 0:
    # Use ToC to differentiate
    pass

# Run function to identify what properties are plain text
ct_by_props = differentiate_pdf_text(doc)
'''
for key in ct_by_props:
    print(key, "==", ct_by_props[key])
'''

# Select body text by whatever set of text props has the largest wordcount
ct, body_text_props = find_body_text(ct_by_props)
font = body_text_props[0]
font_size = body_text_props[1]
font_color_code = body_text_props[2]

print("\nBody text properties:")
print("Font:", font)
print("Font Size:", font_size)
print("Color:", get_rgb_color(font_color_code))
print("Number of words:", ct)
print()

# Go back through and "tag" headers vs. body text



Body text properties:
Font: Calibri-Light
Font Size: 11
Color: rgb(34, 31, 31)
Number of words: 2620

Wall time: 920 ms


In [70]:
def count_words(s):
    return len(findall(r'\w+', s))

# Find properties of body text for a pdf document
def differentiate_pdf_text(doc):
    num_pages = len(doc)
    # Create a dictionary to hold word counts for each set of properties - props:ct
    ct_by_props = {}
        
    for page_num in range(num_pages):          
        # Get page's text (json format)
        json_text = doc[page_num].getText("json")

        # Convert string/json to usable format (dictionary/lists)
        json_text = json.loads(json_text)  
        text_blocks = json_text["blocks"]

        # Set previous properties to None to prepare for comparisons
        prev_props = (None, None, None)
        text_holder = ""

        # Iterate through all the text entries to count words for each set of properties
        for entry in text_blocks: 
            # Check if this block is an image
            try:
                img = entry["image"]
            except:
                img = None
                pass

            # Check if this block exists, or if the end of the page has been reached
            try: 
                lines = entry["lines"]
            except:
                lines = None
                pass

            # Line is not an image and contains text so we continue
            if lines != None and img == None:             
                for spans in lines:
                    # Get the line data
                    spans = spans["spans"]                

                    for i in range(len(spans)):
                        span = spans[i]

                        # Identify current line's properties and text
                        # put into tuple - (font, size, color)
                        props = (span["font"], round(span["size"]), span["color"])
                        text = span["text"]                      

                        ct = count_words(text_holder)

                        # Update dictionary
                        if props not in ct_by_props:
                            ct_by_props[props] = ct
                        else:
                            ct_by_props[props] = ct_by_props[props] + ct

                        # Update text holder & prev properties to new
                        text_holder = text
                        prev_props = props
    return ct_by_props

In [4]:
def find_body_text(ct_by_props):
    max_ct = 0
    max_ct_props = None
    for props in ct_by_props:
        props_ct = ct_by_props[props]
        if props_ct > max_ct:
            max_ct = props_ct
            max_ct_props = props
    return max_ct, max_ct_props

In [5]:
def get_rgb_color(color_code):
    cc = fitz.sRGB_to_rgb(color_code)    
    cc = "".join(str(cc))
    cc = "rgb" + cc
    return cc

cc = get_rgb_color(13027530)
print(cc)

rgb(198, 200, 202)


### Notes section (Misc. code used for debugging)

In [95]:
# Check what style is like for json entries/spans/span/text
doc = fitz.open("./downloaded/DOC.pdf")
page_num = 1

def print_first_entry(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    
    # Iterate through all the text entries
    for entry in text_blocks:
        
        lines = entry["lines"]
        print("LINES:\n", lines, "\n")

        if lines != None:
            for spans in lines:       
                # Get the line data
                spans = spans["spans"]
                
                print("SPANS:\n", spans, "\n")

                for i in range(len(spans)):
                    span = spans[i]
                    
                    print("SPAN:\n", span, "\n")
                    
                    # Identify current line's properties and text
                    text = span["text"]
                    print("TEXT:\n", text, "\n")
        break
    return

print_first_entry(doc,page_num)

mupdf: cannot open ./downloaded/DOC.pdf: No such file or directory


RuntimeError: cannot open ./downloaded/DOC.pdf: No such file or directory

In [94]:
doc = fitz.open("./downloaded/DOC.pdf")
page_num = 1

def get_json_text(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Group json entries by those that share the same color&font&size

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    return print(text_blocks)

get_json_text(doc, page_num)

mupdf: cannot open ./downloaded/DOC.pdf: No such file or directory


RuntimeError: cannot open ./downloaded/DOC.pdf: No such file or directory